In [1]:
pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 16.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.0/358.0 kB 26.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import plotly.io as pio
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.express as px

pio.renderers.default = "notebook"

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spotify") \
    .getOrCreate()


In [4]:
df_raw = spark.read.option("header", True).option("inferSchema", True).csv("spotify.csv")
df_raw.printSchema()
df_raw.show(5)

root
 |-- artist: string (nullable = true)
 |-- song: string (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- year: integer (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: integer (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- genre: string (nullable = true)

+--------------+--------------------+-----------+--------+----+----------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+------------+
|        artist|                song|duration_ms|explicit|year|popularity|dance

In [23]:
df.select("song", "artist", "mode", "key").show(5)

+--------------------+---------------+----+---+
|                song|         artist|mode|key|
+--------------------+---------------+----+---+
|        Shalala Lala|      Vengaboys|   1|  2|
|Independent Women...|Destiny's Child|   0|  6|
|     La Camisa Negra|         Juanes|   0|  6|
|When The Sun Goes...| Arctic Monkeys|   0| 11|
|Blah Blah Blah (f...|          Kesha|   1| 10|
+--------------------+---------------+----+---+
only showing top 5 rows



# 🎧 Dataset Spotify – Dicionário de Variáveis

Este notebook utiliza um dataset de músicas do Spotify contendo diversas informações musicais, técnicas e contextuais de cada faixa. Abaixo está o dicionário das variáveis presentes no conjunto de dados:

## 🧾 Informações Gerais

- **`artist`**: Nome do(a) artista da música.
- **`song`**: Título da faixa musical.
- **`genre`**: Gênero musical da faixa.
- **`year`**: Ano de lançamento da música.

## ⏱️ Duração e Popularidade

- **`duration_ms`**: Duração da faixa em milissegundos.
- **`explicit`**: Indica se a música contém conteúdo explícito (1 = sim, 0 = não).
- **`popularity`**: Grau de popularidade da música no Spotify. Valores mais altos indicam maior popularidade.

## 🎶 Atributos Musicais

- **`danceability`**: Indica quão dançante é a faixa, de 0.0 (menos dançável) a 1.0 (mais dançável).
- **`energy`**: Medida da intensidade e atividade da faixa, variando de 0.0 a 1.0.
- **`key`**: Tom musical da música (0 = C, 1 = C♯/D♭, ..., 11 = B). Se não detectado, o valor é -1.
- **`mode`**: Modalidade da faixa: 1 = maior, 0 = menor.
- **`tempo`**: Tempo estimado da música, em batidas por minuto (BPM).
- **`loudness`**: Volume médio da faixa em decibéis (dB), normalmente entre -60 e 0.

## 🔍 Análise de Conteúdo

- **`speechiness`**: Indica a presença de fala na faixa. Valores próximos a 1.0 indicam faixas faladas.
- **`acousticness`**: Confiança de que a faixa é acústica (0.0 a 1.0).
- **`instrumentalness`**: Probabilidade da faixa ser instrumental. Valores acima de 0.5 indicam instrumental.
- **`liveness`**: Probabilidade de a faixa ter sido gravada ao vivo. Valores > 0.8 indicam alta chance.
- **`valence`**: Positividade emocional transmitida pela faixa. Valores altos indicam músicas mais alegres.

---

> Este dicionário serve como referência para as análises exploratórias, seleção de features e aplicação de algoritmos de Machine Learning ao longo do notebook.


In [5]:
df = df_raw

In [6]:
total_registros = df.count()

# Contar número de registros distintos
total_distintos = df.dropDuplicates().count()

# Número de duplicatas
duplicatas = total_registros - total_distintos

print(f"Total de registros: {total_registros}")
print(f"Registros distintos: {total_distintos}")
print(f"Registros duplicados: {duplicatas}")

Total de registros: 2000
Registros distintos: 1941
Registros duplicados: 59


In [8]:
df = df.dropDuplicates()
print(df.count())

1941


In [9]:
import plotly.graph_objects as go
import numpy as np

In [10]:
colunas_numericas = ["danceability", "energy", "popularity", "duration_ms", "explicit", "loudness", "key"]


In [12]:
from pyspark.sql.functions import col
df = df.withColumn("explicit",col("explicit").cast("int"))

In [13]:
matriz_correlacao = []
for i in colunas_numericas:
    linha = []
    for j in colunas_numericas:
        corr = df.stat.corr(i,j)
        linha.append(corr)
    matriz_correlacao.append(linha)
    

In [14]:
matriz_correlacao = np.array(matriz_correlacao)

In [19]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
    z=matriz_correlacao,
    x=colunas_numericas,
    y=colunas_numericas,
    colorscale='RdBu',
    zmin=-1,
    zmax=-1,
    text=[[f"{val:.2f}" for val in linha] for linha in matriz_correlacao],
    texttemplate="%{text}",
    hoverinfo="text"
))
fig.update_layout(
    title="Matriz de Correlação",
    xaxis=dict(tickangle=45),
    height=800,
    width=800
)
fig.show()

In [20]:
df_pd = df.select(*colunas_numericas).dropna().toPandas()

In [21]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
from scipy.stats import gaussian_kde

# Lista de colunas numéricas (sem 'explicit')
colunas_plot = ["danceability", "energy", "popularity", "duration_ms", "loudness", "key"]

# Converter para Pandas
df_pd = df.select(colunas_plot).dropna().toPandas()

# Criar subplots em pares (2 colunas)
linhas = (len(colunas_plot) + 1) // 2
fig = make_subplots(rows=linhas, cols=2, subplot_titles=[col.capitalize() for col in colunas_plot])

# Adicionar histogramas + KDE
for idx, coluna in enumerate(colunas_plot):
    row = idx // 2 + 1
    col = idx % 2 + 1

    valores = df_pd[coluna].dropna()
    hist = go.Histogram(x=valores, nbinsx=30, name="Histograma", opacity=0.6, marker=dict(color="lightblue"), showlegend=False)
    kde = gaussian_kde(valores)
    x_vals = np.linspace(valores.min(), valores.max(), 200)
    y_vals = kde(x_vals) * len(valores) * (x_vals[1] - x_vals[0])  # ajusta escala

    linha_kde = go.Scatter(x=x_vals, y=y_vals, name="KDE", line=dict(color="darkblue"), showlegend=False)

    fig.add_trace(hist, row=row, col=col)
    fig.add_trace(linha_kde, row=row, col=col)

# Layout geral
fig.update_layout(
    height=300 * linhas,
    width=900,
    title_text="📊 Distribuição de Variáveis Numéricas com Curva de Densidade",
    showlegend=False
)

fig.show()


In [24]:
import plotly.graph_objects as go
import pandas as pd

# Supondo que a coluna 'genre' está presente no DataFrame
df_genres = df.select("genre").groupBy("genre").count().orderBy("count", ascending=False).toPandas()

# Gráfico de barras
bar = go.Bar(
    x=df_genres["genre"],
    y=df_genres["count"],
    name="Total de músicas",
    marker_color="limegreen"
)

# Boxplot (sobre os mesmos dados para ilustrar variação)
box = go.Box(
    y=df_genres["count"],
    name="Distribuição",
    boxpoints=False,
    marker_color="green"
)

# Figura combinada
fig = go.Figure()

fig.add_trace(bar)
fig.add_trace(box)

fig.update_layout(
    title="🎼 Total de Músicas por Gênero",
    xaxis_title="Gênero",
    yaxis_title="Quantidade de Músicas",
    xaxis_tickangle=45,
    template="plotly_dark",
    width=1000,
    height=600,
    showlegend=False
)

fig.show()

In [25]:
import plotly.express as px

# Agrupar número de músicas por artista
df_artistas = df_raw.groupBy("artist").count().orderBy("count", ascending=False).toPandas()

# Gerar TreeMap
fig = px.treemap(
    df_artistas,
    path=["artist"],
    values="count",
    title="🎶 TreeMap of Singers Playlist",
    color="count",
    color_continuous_scale="Blues"
)

fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
fig.show()

In [26]:
import plotly.express as px

# Garantir que 'explicit' esteja em formato booleano ou string legível
df_box = df.select("popularity", "explicit").dropna().toPandas()
df_box["explicit"] = df_box["explicit"].map({0: "False", 1: "True", False: "False", True: "True"})

# Criar o boxplot
fig = px.box(
    df_box,
    x="explicit",
    y="popularity",
    color="explicit",
    color_discrete_map={"False": "cyan", "True": "magenta"},
    title="📦 Popularity Based on Explicit Content",
)

fig.update_layout(
    xaxis_title="Explicit",
    yaxis_title="Popularity",
    template="plotly_dark",
    showlegend=True
)

fig.show()


In [27]:
import plotly.express as px

# Selecionar e converter os dados
df_speech = df.select("speechiness", "popularity").dropna().toPandas()

# Criar o gráfico
fig = px.scatter(
    df_speech,
    x="speechiness",
    y="popularity",
    color="speechiness",
    color_continuous_scale="plasma",
    title="🗣️ Speechiness Versus Popularity"
)

fig.update_layout(
    xaxis_title="Speechiness",
    yaxis_title="Popularity",
    template="plotly_dark"
)

fig.show()

In [28]:
features_kmeans = [
    "danceability",
    "energy",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "valence",
    "tempo",
    "duration_ms"
]


In [29]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler

In [30]:
assembler = VectorAssembler(
    inputCols= features_kmeans,
    outputCol="features_raw"
)

In [31]:
scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withMean=True,
    withStd=True,
)

In [32]:
prep_pipeline = Pipeline(stages=[assembler,scaler])

In [33]:
df_preparado = prep_pipeline.fit(df.select(*features_kmeans).dropna()).transform(df)

In [35]:
df_preparado.select("features").show(5,truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                           |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[0.5916151480538784,1.1738693953388621,-0.7382650287980222,-0.4506369653550664,-0.13910020245478197,1.9019362996833509,0.14295998962222486,-0.3509820455835737]    |
|[0.4422640540030394,-0.7820201540904486,1.0631275332467391,1.3548625142471014,-0.1739113634705034,1.693645777166487,-0.8226769784701359,-0.1901149678675613]       |
|[0.5916151480538784,0.06182516991082583,-0.759066328590687,-0.25710781702800484,-0.17395311907515573,1.9019362996833509,-0.8577634308656181,-0.3029053627121585]   |
|[-2

In [45]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

avaliador = ClusteringEvaluator(
    featuresCol="features",
    predictionCol="prediction",
    metricName="silhouette",
    distanceMeasure="squaredEuclidean"
)

In [50]:
k_values = list(range(2,11))
resultados = []

In [51]:
for k in k_values:
    kmeans = KMeans(k=k,seed=42,featuresCol="features", predictionCol="prediction") 
    pipeline_k = Pipeline(stages=prep_pipeline.getStages() + [kmeans]) 
    modelo_k = pipeline_k.fit(df.select(*features_kmeans).dropna())
    resultado_k= modelo_k.transform(df.select(*features_kmeans).dropna())
    silhoutte = avaliador.evaluate(resultado_k)
    wssse = modelo_k.stages[-1].summary.trainingCost
    resultados.append((k,silhoutte,wssse))
    print(f"k={k} | Silhouette: {silhoutte:.4f} | WSSE: {wssse:.2f}")

k=2 | Silhouette: 0.2722 | WSSE: 13494.58
k=3 | Silhouette: 0.2463 | WSSE: 12004.76
k=4 | Silhouette: 0.2616 | WSSE: 10426.20
k=5 | Silhouette: 0.2764 | WSSE: 9256.47
k=6 | Silhouette: 0.2694 | WSSE: 8536.26
k=7 | Silhouette: 0.2705 | WSSE: 8192.75
k=8 | Silhouette: 0.2712 | WSSE: 7962.76
k=9 | Silhouette: 0.2353 | WSSE: 7280.73
k=10 | Silhouette: 0.1275 | WSSE: 8382.09


In [52]:
import plotly.graph_objects as go

# Separar resultados
ks = [r[0] for r in resultados]
silhouettes = [r[1] for r in resultados]
wssses = [r[2] for r in resultados]

# Gráfico com duas y-axes
fig = go.Figure()

fig.add_trace(go.Scatter(x=ks, y=wssses, mode='lines+markers', name='WSSSE', yaxis='y1'))
fig.add_trace(go.Scatter(x=ks, y=silhouettes, mode='lines+markers', name='Silhouette Score', yaxis='y2'))

fig.update_layout(
    title="📈 Avaliação de k (Elbow + Silhouette)",
    xaxis=dict(title='Número de Clusters (k)'),
    yaxis=dict(title='WSSSE', side='left'),
    yaxis2=dict(title='Silhouette Score', overlaying='y', side='right'),
    legend=dict(x=0.5, xanchor="center", y=1.15, orientation="h"),
    width=800,
    height=500
)

fig.show()


In [66]:
kmeans_final = KMeans(k=5,seed=42,featuresCol="features", predictionCol="cluster") 
pipeline_final = Pipeline(stages=prep_pipeline.getStages() + [kmeans_final]) 
modelo_final = pipeline_final.fit(df.select(*features_kmeans).dropna())
resultado_final = modelo_final.transform(df.select(*features_kmeans).dropna())

In [70]:
resultado_final.select(*features_kmeans,"cluster").show(5)

+------------+------+-----------+------------+----------------+-------+-------+-----------+-------+
|danceability|energy|speechiness|acousticness|instrumentalness|valence|  tempo|duration_ms|cluster|
+------------+------+-----------+------------+----------------+-------+-------+-----------+-------+
|       0.751| 0.901|     0.0328|      0.0504|         0.00308|  0.973|124.017|     214819|      1|
|        0.73| 0.602|      0.206|       0.362|         3.69E-6|  0.927| 97.954|     221133|      4|
|       0.751| 0.731|     0.0308|      0.0838|             0.0|  0.973| 97.007|     216706|      1|
|       0.348| 0.875|      0.199|      0.0341|             0.0|  0.407|169.152|     202133|      3|
|       0.752| 0.836|      0.115|      0.0843|         4.25E-4|  0.519|120.003|     172053|      1|
+------------+------+-----------+------------+----------------+-------+-------+-----------+-------+
only showing top 5 rows



In [71]:
# Selecionar dados e converter para Pandas
variaveis = features_kmeans
df_cluster_plot = resultado_final.select(variaveis + ["cluster"]).toPandas()
df_cluster_plot["cluster"] = df_cluster_plot["cluster"].astype(str)

# Gerar boxplots de cada variável por cluster
for var in variaveis:
    fig = px.box(
        df_cluster_plot,
        x="cluster",
        y=var,
        color="cluster",
        title=f"📦 Distribuição de {var} por Cluster",
        labels={"cluster": "Cluster", var: var.capitalize()}
    )
    fig.update_layout(template="plotly_dark", width=800, height=500)
    fig.show()


## 🧬 Perfis dos Clusters com Base em Métricas Musicais

Abaixo estão as interpretações dos 5 clusters formados pelo KMeans, considerando tanto **médias quanto medianas** das variáveis musicais. Essa análise ajuda a entender o "estilo" predominante em cada grupo.

---

### 🎧 Cluster 0 – **Moderadamente dançante e acústico**
- `danceability`: média 0.63 – **nível moderado**
- `energy`: média 0.51 – **baixo para moderado**
- `acousticness`: 0.44 – **acústico balanceado**
- `instrumentalness`: muito baixo
- `valence`: 0.39 – **ambiente emocional mais melancólico**
- `tempo`: 109 BPM – **ritmo mais calmo**
- **Duração média mais alta (232s)**

📌 **Perfil:** músicas calmas, mais acústicas, possivelmente baladas ou faixas suaves.

---

### 💃 Cluster 1 – **Altamente dançante e feliz**
- `danceability`: média 0.74 – **mais alto de todos**
- `energy`: média 0.77 – **muito energético**
- `acousticness`: muito baixo (~0.08)
- `valence`: 0.72 – **clima alegre**
- `tempo`: 115 BPM – **ritmo animado**
- **Duração mais curta**

📌 **Perfil:** faixas pop/dançantes, alegres, voltadas para festas e rádio.

---

### 🎹 Cluster 2 – **Instrumental e positivo**
- `instrumentalness`: 0.67 – **claramente instrumental**
- `acousticness`: 0.13 – mais eletrônico
- `valence`: 0.52 – neutro a positivo
- `tempo`: 124 BPM – ritmo animado

📌 **Perfil:** trilhas instrumentais eletrônicas ou dançantes, sem vocais, mas com clima animado.

---

### 🎼 Cluster 3 – **Mais lento e neutro**
- `danceability`: 0.56 – o **menor**
- `valence`: 0.39 – baixo, ambiente mais emocional
- `tempo`: 128 BPM – ritmo alto, mas possivelmente irregular
- `acousticness`: muito baixo

📌 **Perfil:** músicas eletrônicas ou experimentais mais densas e menos dançantes, talvez com construções complexas.

---

### 🗣️ Cluster 4 – **Músicas com fala (speechiness elevado)**
- `speechiness`: 0.29 – **muito acima dos outros clusters**
- `valence`: 0.58 – positivo
- `danceability`: 0.71 – alto
- `energy`: 0.69 – bom equilíbrio

📌 **Perfil:** faixas faladas, possivelmente **rap, spoken word ou faixas com trechos narrados**, mantendo caráter dançante e positivo.

---

## 🧠 Conclusão:

Cada cluster representa um **subconjunto distinto de músicas** com características sonoras próprias. Destaques:

- Clusters 1 e 2 são os mais dançantes, com emoções positivas.
- Cluster 2 se diferencia por ser **instrumental**.
- Cluster 4 é o mais associado a **conteúdo falado**.
- Cluster 3 representa o grupo com menor dançabilidade e valência.
- Cluster 0 é mais **acústico, emocional e longo** – talvez o mais "introspectivo".

> Essa segmentação pode ser usada para criar playlists temáticas, ajustar recomendações ou entender padrões musicais de artistas e gêneros.


In [74]:
from pyspark.sql.functions import avg

# 📊 Calcular as médias por cluster
df_medias = resultado_final.groupBy("cluster").agg(
    *[avg(c).alias(c) for c in features_kmeans]
).orderBy("cluster")

# 📤 Coletar os valores para uso em Plotly
medias_clusters = df_medias.select(*features_kmeans).rdd.map(lambda row: [float(x) for x in row]).collect()

# 👀 Verificar estrutura gerada
for i, linha in enumerate(medias_clusters):
    print(f"Cluster {i}: {linha}")

Cluster 0: [0.6316377952755907, 0.5126023622047243, 0.07955196850393702, 0.44020358267716525, 0.0033713337007874014, 0.3999086614173229, 109.09595275590549, 232148.52755905513]
Cluster 1: [0.7428779840848801, 0.7733328912466848, 0.0730692307692308, 0.08680828289124669, 0.005972297745358092, 0.7252652519893892, 115.77621618037128, 219106.24535809018]
Cluster 2: [0.6853666666666666, 0.7377, 0.07010666666666668, 0.13350831999999999, 0.6681333333333334, 0.5249033333333333, 123.96089999999998, 219950.76666666666]
Cluster 3: [0.5658534201954393, 0.7571319218241045, 0.06495993485342023, 0.057205957654723145, 0.006661052312703583, 0.39069674267101007, 128.7561482084691, 233489.67589576548]
Cluster 4: [0.7185674740484425, 0.6928096885813154, 0.29118685121107246, 0.11207006920415225, 0.0011925867128027682, 0.5856249134948096, 122.65326989619385, 240726.03806228374]


In [75]:
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 📌 Variáveis usadas no clustering (mesma ordem do pipeline)
variaveis = [
    "danceability", "energy", "speechiness", "acousticness",
    "instrumentalness", "valence", "tempo", "duration_ms"
]

# 🧼 Normalizar os dados para escala 0-1
dados_numericos = np.array(medias_clusters)
dados_escalados = MinMaxScaler().fit_transform(dados_numericos)

# 🕸️ Construir radar plot
fig = go.Figure()

for i, linha in enumerate(dados_escalados):
    fig.add_trace(go.Scatterpolar(
        r=linha,
        theta=variaveis,
        fill='toself',
        name=f"Cluster {i}"
    ))

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    title="📊 Perfil Musical Médio por Cluster (Radar Plot)",
    showlegend=True,
    width=800,
    height=600
)

fig.show()


In [88]:
from pyspark.ml.feature import PCA
pca = PCA(k=3,inputCol="features",outputCol="pca_features")
modelo_pca = pca.fit(resultado_final)

In [89]:
df_pca = modelo_pca.transform(resultado_final)

In [90]:
df_pca.select("pca_features").show(truncate=False)

+---------------------------------------------------------------+
|pca_features                                                   |
+---------------------------------------------------------------+
|[-2.124746847133842,-0.33779135009711514,0.6594523418879793]   |
|[0.1446405050562397,-2.1748465673255177,-0.21280536763894561]  |
|[-1.2385973843375107,-1.1853292084616363,0.9799277392460093]   |
|[-0.3194653557255508,2.670251547917424,-1.2064681270865079]    |
|[-0.9010298283929034,-0.13134605723595624,0.49565123788652266] |
|[-0.7635114651107996,-0.9600241296557676,0.10460220771251427]  |
|[-0.5633048798079067,1.411801682892871,0.42703156337634907]    |
|[0.2383118368707664,2.333068753634656,0.25114399407167814]     |
|[-1.1119696823024385,-0.7057269246458852,-0.0536896530236517]  |
|[-1.4675758050522543,-0.18137698531196533,1.3909656598320266]  |
|[2.9828069393428676,-1.6458205870800455,-1.4452824693706918]   |
|[-1.2424609595814853,-0.1984983823389935,-2.299932387411331]   |
|[0.626735

In [94]:
df_plot = df_pca.select("pca_features", "cluster").toPandas()
df_plot["cluster"] = df_plot["cluster"].astype(str)
df_plot["x"] = df_plot["pca_features"].apply(lambda v: float(v[0]))
df_plot["y"] = df_plot["pca_features"].apply(lambda v: float(v[1]))
df_plot["z"] = df_plot["pca_features"].apply(lambda v: float(v[2]))


# 📈 Scatter Plot
fig = px.scatter_3d(
    df_plot,
    x="x",
    y="y",
    z="z",
    color="cluster",
    title="🎯 Visualização dos Clusters com PCA 2D",
    labels={"x": "Componente Principal 1", "y": "Componente Principal 2"},
    opacity=0.7
)

fig.update_layout(template="plotly_dark", width=800, height=600)
fig.show()

In [85]:
df_features = prep_pipeline.fit(df.select(*features_kmeans).dropna()).transform(df.select(*features_kmeans).dropna())
df_com_cluster = modelo_final.transform(df.select(*features_kmeans).dropna())


In [87]:
from pyspark.sql.functions import monotonically_increasing_id
df_raw_indexed = df_raw.withColumn("idx", monotonically_increasing_id())
df_cluster_indexed = df_com_cluster.withColumn("idx", monotonically_increasing_id())
df_features_indexed = df_features.withColumn("idx", monotonically_increasing_id())

# Fazer join por índice para juntar tudo
df_final = df_raw_indexed.join(df_cluster_indexed.select("cluster", "idx"), on="idx") \
                        .join(df_features_indexed.select("features", "idx"), on="idx") \
                        .drop("idx")

# Mostrar resultado final
df_final.show(5, truncate=False)

+--------------+----------------------+-----------+--------+----+----------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+------------+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|artist        |song                  |duration_ms|explicit|year|popularity|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo  |genre       |cluster|features                                                                                                                                                           |
+--------------+----------------------+-----------+--------+----+----------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+------------+-------+----------------------------------------------------------------

In [105]:
df_final.show(1)

+--------------+--------------------+-----------+--------+----+----------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+------+-----+-------+--------------------+
|        artist|                song|duration_ms|explicit|year|popularity|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence| tempo|genre|cluster|            features|
+--------------+--------------------+-----------+--------+----+----------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+------+-----+-------+--------------------+
|Britney Spears|Oops!...I Did It ...|     211160|   false|2000|        77|       0.751| 0.834|  1|  -5.444|   0|     0.0437|         0.3|         1.77E-5|   0.355|  0.894|95.053|  pop|      1|[0.59161514805387...|
+--------------+--------------------+-----------+--------+----+----------+------------+------+---+--------+----+-----------+------------+-------

In [96]:
from pyspark.ml.feature import BucketedRandomProjectionLSH
modelos_lsh_por_cluster = {}

In [98]:
clusters = [row["cluster"] for row in df_final.select("cluster").distinct().collect()]
clusters

[1, 3, 4, 2, 0]

In [104]:
df_c = df_final.filter(col("cluster") == 0 )


TypeError: 'int' object is not callable

In [107]:
from pyspark.sql.functions import col

for c in clusters:
    df_c = df_final.filter(col("cluster") == c)
    brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=5.0, numHashTables=3)
    modelo_lsh = brp.fit(df_c)
    modelos_lsh_por_cluster[c] = modelo_lsh
print(f"Modelos LSH para clusters: {list(modelos_lsh_por_cluster.keys())}" )

Modelos LSH para clusters: [1, 3, 4, 2, 0]


In [116]:
import random
def recomendar_musicas(df_final, modelos_lsh_por_cluster,k=3):
    total = df_final.count()
    indice_aleatorio = random.randint(0,total-1)
    musica = df_final.limit(indice_aleatorio + 1).collect()[-1]
    print(f"Música sorteada: {musica['artist']} - {musica['song']} (Cluster {musica['cluster']})")

    cluster_musica = musica["cluster"]
    vetor_musica = musica["features"]
    modelo_lsh = modelos_lsh_por_cluster[cluster_musica]
    recomendacoes = modelo_lsh.approxNearestNeighbors(df_final.filter(col("cluster") == cluster_musica), vetor_musica, k + 1) \
        .filter(~((col("artist") == musica["artist"]) & (col("song") == musica["song"]))) \
        .limit(k)
    recomendacoes.select("artist", "song", "distCol").show(truncate=False)
    return recomendacoes


In [128]:
recomendar_musicas(df_final,modelos_lsh_por_cluster)

Música sorteada: Enrique Iglesias - Bailando - Spanish Version (Cluster 4)
+--------+--------------------------+------------------+
|artist  |song                      |distCol           |
+--------+--------------------------+------------------+
|Kesha   |We R Who We R             |0.8071786898322355|
|Train   |Drops of Jupiter (Tell Me)|0.858890568757023 |
|DJ Fresh|Gold Dust - Radio Edit    |1.0436041974632309|
+--------+--------------------------+------------------+



DataFrame[artist: string, song: string, duration_ms: int, explicit: boolean, year: int, popularity: int, danceability: double, energy: double, key: int, loudness: double, mode: int, speechiness: double, acousticness: double, instrumentalness: double, liveness: double, valence: double, tempo: double, genre: string, cluster: int, features: vector, hashes: array<vector>, distCol: double]